# 개,고양이 이미지 분류
데이터 : https://www.kaggle.com/competitions/dogs-vs-cats/data?select=train.zip

In [181]:
import numpy as np
import pandas as pd
import os,shutil,pathlib

In [183]:
dir1=pathlib.Path('data/train')
dir2=pathlib.Path('data/holdout')

In [190]:
def make_data(name,s_idx,e_idx):
    for cate in ('cat','dog'):
        dir=dir2/name/cate
        os.makedirs(dir)
        fnames=[f'{cate}.{i}.jpg'
               for i in range(s_idx,e_idx)]
        for fname in fnames:
            shutil.copyfile(src=dir1/fname,
                           dst=dir/fname)

In [193]:
make_data('train',s_idx=0,e_idx=1000)
make_data('valid',s_idx=1000,e_idx=1500)
make_data('test',s_idx=1500,e_idx=2500)

## 모델 생성

In [176]:
from tensorflow import keras
from tensorflow.keras import layers

In [177]:
model=keras.Sequential()
model.add(keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',
                             input_shape=(180,180,3)))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [178]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 89, 89, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 20, 20, 128)     

In [179]:
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics='accuracy')

## 이미지 읽기

In [194]:
from tensorflow.keras.utils import image_dataset_from_directory

tr_dt=image_dataset_from_directory(
    dir2/'train',
    image_size=(180,180),
    batch_size=32)

val_dt=image_dataset_from_directory(
    dir2/'valid',
    image_size=(180,180),
    batch_size=32)

te_dt=image_dataset_from_directory(
    dir2/'test',
    image_size=(180,180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [197]:
callbacks=[
    keras.callbacks.ModelCheckpoint(
    filepath='convnet.keras',
    save_best_only=True,
    monitor='val_loss'),
    keras.callbacks.EarlyStopping(patience=3)
]

In [198]:
his=model.fit(tr_dt,epochs=20,validation_data=val_dt,callbacks=callbacks)

Epoch 1/20


ValueError: Creating variables on a non-first call to a function decorated with tf.function.